In [ ]:
import pandas as pd
from pathlib import Path
import json
import gzip
import glob

In [ ]:
DATA_FOLDER = "data"
CHARGE_FOLDER = "charge"
STATIONS_FOLDER = "stations"

In [ ]:
df_charge = pd.read_parquet(Path("data/charge/"), engine="pyarrow")

In [ ]:
# Load the newest file in the folder ()

stations_path = Path(DATA_FOLDER) / STATIONS_FOLDER 
static_path = Path(DATA_FOLDER) / STATIONS_FOLDER
files = glob.glob(str(static_path / "stations_*.json.gz"))
latest_file = max(files, key=lambda x: Path(x).stat().st_mtime)

with gzip.open(latest_file, "rt", encoding="utf-8") as file:
	stations = json.load(file)["EVSEData"]

In [ ]:
# Series of operator IDs to names
ser_operators = pd.Series({operator['OperatorID']: operator['OperatorName'] for operator in stations})

In [ ]:
# A row for each station
rows = []
for operator in stations:
    for station in operator['EVSEDataRecord']:
        row = station
        row['Operator'] = operator['OperatorID']
        rows.append(row)
df_stations = pd.DataFrame(rows)

In [ ]:
df_stations.columns

In [ ]:
# Pivot (might be heavy on memory)
#df_data_pvt = df_data.pivot(index='TIME', columns='STATION_ID', values='STATUS')
#df_data_pvt.head()